In [13]:
import tabula

# 2022 Report

In [16]:
url = "https://portal.ct.gov/-/media/CID/1_Reports/2022-ConsumerReportCard.pdf"
pages = [7, 8] + list(range(29, 49 + 1)) + [52, 55, 56, 57, 58, 59]
dfs = tabula.read_pdf(url, stream=True, pages=[7, 8, 29])